## Part 2 MCDA

In [1]:
import os
import topogenesis as tg
import trimesh as tm
import numpy as np
import pyvista as pv
import panel as pn
import pandas as pd 

from ladybug.sunpath import Sunpath #ladybug sunpath 호출
from IPython.display import display, Image

In [2]:
csv_path = os.path.relpath('original_lattice/interior_lattice.csv')

In [3]:
# tri_to_pv 함수 정의: 삼각형 메시(tri_mesh)를 PyVista 형식으로 변환합니다.
def tri_to_pv(tri_mesh):
    # tri_mesh.faces 배열에 각 face의 시작 부분에 3을 추가하여 새로운 배열을 만듭니다.
    # 'constant' 모드로 패딩을 추가하여 각 face의 시작에 3을 삽입합니다.
    faces = np.pad(tri_mesh.faces, ((0, 0), (1, 0)), 'constant', constant_values=3)
    # PyVista의 PolyData 객체를 생성하여, tri_mesh의 정점(vertices)과 패딩된 faces 배열을 사용합니다.
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    
    # 변환된 PyVista 메쉬 객체를 반환합니다.
    return pv_mesh

In [4]:
context_mesh = tm.load(os.path.relpath('obj\context.obj'))  # 주변환경 obj 메시 불러오기
street_pc = tg.cloud_from_csv("pts\pts_str(11).CSV")        # street point as csv (포인트를 csv로 저장) 
entrance_pc = tg.cloud_from_csv("pts\ent_pts01.CSV")       # entrance point as csv (라이노 포인트를 csv로 저장)
envelope_lattice = tg.lattice_from_csv(csv_path)

In [5]:
# creating neighborhood definition 1
stencil = tg.create_stencil("von_neumann", 1, 1)
# setting the indices
stencil.set_index([0,0,0], 0)

# creating neighborhood definition 1
stencil_sq = tg.create_stencil("von_neumann", 1, 1)
# setting the indices
stencil_sq.set_index([0,0,0], 0)
stencil_sq.set_index([0,0,1], 0)
stencil_sq.set_index([0,0,-1], 0)

print(stencil_sq)


# 'von Neumann' 정사각형 스텐실 생성

avail_lattice = tg.lattice_from_csv(csv_path)                                         # CSV 파일에서 격자 데이터 불러오기
init_avail_lattice = tg.to_lattice(np.copy(avail_lattice), avail_lattice)             # 복사된 데이터를 격자 객체로 변환

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [0 0 0]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


In [6]:
#df = pd.read_excel('excel/program (오피스).xlsx', sheet_name='Sheet1')
df = pd.read_excel('excel/program (이노션).xlsx', sheet_name='Sheet1')
#df = pd.read_excel('excel/program (학교).xlsx', sheet_name='Sheet1')
#df = pd.read_excel('excel/program (시니어 센터).xlsx', sheet_name='Sheet1')

program_prefs = df[['sun_acc', 'ent_acc', 'str_acc','ung_pre','top_pre']].copy()
program_prefs.reset_index(drop=True, inplace=True)
#print(program_prefs)

In [7]:
space_list = df['space_name'].to_dict()
print(space_list)

{0: 'Lobby', 1: 'Branding Space', 2: 'External Contact', 3: 'Interview Room', 4: 'Personal Workspace', 5: 'Work Support Area', 6: 'Personal Locker Room', 7: 'Executive Space', 8: 'VIP Reception', 9: 'Secretary Room', 10: 'Pantry', 11: 'Executive Meeting', 12: 'Executive Offices', 13: 'Focus Rooms', 14: 'Project Rooms', 15: 'Conference Rooms', 16: '4-Person Meeting', 17: '6-Person Meeting', 18: '8-Person Meeting', 19: '12-Person Meeting', 20: '18-Person Meeting', 21: 'Idea Room', 22: 'Mail Room', 23: 'Oasis', 24: 'PT Practice Room', 25: 'Screening Room', 26: 'Studio', 27: 'Video Editing', 28: 'Plotter/Board Room', 29: 'Training Room', 30: 'Ino Kitchen', 31: 'Kitchen', 32: 'Ino Cafe', 33: 'Fitness Facilities', 34: 'Dressing/Shower', 35: 'Counseling Room', 36: 'Personal Rest Room', 37: 'Sleeping Room', 38: 'Nursing Room', 39: 'Work Lounge', 40: 'Town Hall', 41: 'Seminar Room', 42: 'Library', 43: 'Social Space', 44: 'Outdoor Terrace', 45: "Driver's Waiting", 46: 'Server Room', 47: 'Cleanin

In [8]:
sizes_complete = df['vox_amount']
agent_areas = []                                                                      # 'agent_areas' 리스트 초기화
for area in sizes_complete:
  agent_areas.append(round(int(area)))

print(agent_areas)

[500, 250, 333, 50, 5000, 300, 100, 50, 40, 20, 20, 60, 500, 100, 200, 100, 8, 15, 20, 30, 45, 50, 10, 50, 20, 50, 30, 40, 30, 60, 100, 300, 116, 150, 50, 20, 20, 40, 20, 300, 200, 200, 100, 150, 100, 20, 30, 50, 30, 40, 50]


In [9]:
fields = {}                                                                           # 필드를 저장할 딕셔너리 초기화
for f in program_prefs.columns:                                                       # 프로그램 선호도의 각 열에 대해 반복
    lattice_path = os.path.relpath('matrixs/' + f + '.csv')                           # 각 필드에 대한 격자 파일 경로 설정
    try:
        fields[f] = tg.lattice_from_csv(lattice_path)                                 # 격자 파일에서 데이터 불러오기
    except:
        fields[f] = copy.deepcopy(avail_lattice * 0 + 1)                              # 파일이 없을 경우, 기본 격자 데이터로 새 격자 생성

top_pre_index = df.columns.get_loc('top_pre')
program_mtx = df.iloc[:, top_pre_index + 1:]
program_mtx.columns = range(len(program_mtx.columns))

program_prefs_copy = program_prefs.copy()
program_prefs_copy['space_id'] = df.index
program_prefs_copy.reset_index(drop=True, inplace=True)
program_complete = program_prefs_copy
# Print the updated 'program_prefs_copy' DataFrame
print(program_mtx)

     0    1    2    3    4    5    6    7    8    9   ...   41   42   43   44  \
0   1.0  0.9  1.0  0.7  0.7  0.1  0.1  0.3  0.6  0.6  ...  0.7  0.5  0.6  0.1   
1   0.9  1.0  0.1  1.0  0.3  0.1  0.1  0.1  0.6  0.5  ...  0.1  0.6  0.1  0.1   
2   1.0  0.1  1.0  0.3  1.0  0.1  0.1  0.3  0.5  0.1  ...  0.1  0.1  0.1  0.1   
3   0.7  1.0  0.3  1.0  0.1  1.0  1.0  1.0  0.1  0.1  ...  0.3  0.4  0.6  0.5   
4   0.7  0.3  1.0  0.1  1.0  1.0  1.0  0.5  0.1  0.1  ...  0.4  0.6  0.6  0.2   
5   0.1  0.1  0.1  1.0  1.0  1.0  0.5  1.0  0.1  0.1  ...  0.4  0.6  0.2  0.1   
6   0.1  0.1  0.1  1.0  1.0  0.5  1.0  0.1  1.0  1.0  ...  0.4  0.1  0.1  0.1   
7   0.3  0.1  0.3  1.0  0.5  1.0  0.1  1.0  1.0  1.0  ...  0.1  0.1  0.1  0.1   
8   0.6  0.6  0.5  0.1  0.1  0.1  1.0  1.0  1.0  1.0  ...  0.1  0.1  0.1  0.1   
9   0.6  0.5  0.1  0.1  0.1  0.1  1.0  1.0  1.0  1.0  ...  0.1  0.1  0.1  0.1   
10  0.1  0.1  0.1  0.1  0.1  0.1  1.0  1.0  1.0  0.1  ...  0.1  0.1  0.1  0.1   
11  0.1  0.1  0.1  0.1  0.1 

In [10]:
def initialize_agents(avail_lattice, program_prefs, fields):   
    # initialize the occupation lattice
    occ_lattice = avail_lattice * 0 - 1

    # create a list for agents locations
    agn_locs = []
    # for each agent origin ... 
    for a_id, a_prefs in program_prefs.iterrows():
        # create a preference lattice
        pref_lattice = (avail_lattice * 0.0 + 1.0) * avail_lattice
        # choosing three available voxels
        for f, w in a_prefs.iteritems():
            pref_lattice *= fields[f] ** w

        select_id = np.argmax(pref_lattice)
        a_origin_1 = np.unravel_index(select_id, avail_lattice.shape)
        # set the initialised agent to ground level, than later there will be no floating parts
        a_origin_2 = (a_origin_1[0], a_origin_1[1], 1)
        # to prevent voxels from occupying the same place
        for n in range(5):
            if avail_lattice[a_origin_2] == 0:
                a_origin_2 = (a_origin_1[0], a_origin_1[1], 1+n)

        # add the origin to the list of agent locations
        agn_locs.append([a_origin_2])

        # set the origin in availablity lattice as 0 (UNavailable)
        avail_lattice[a_origin_2] = 0

        # set the origin in occupation lattice as the agent id (a_id)
        occ_lattice[a_origin_2] = a_id

    return occ_lattice, agn_locs

In [12]:
def find_optimal_location_for_agent(agent_id, agent_prefs, avail_lattice, fields, stencil, occ_lattice):
    """에이전트에 대한 최적의 시작 위치 찾기"""
    avail_index = np.array(np.where(avail_lattice)).T
    a_eval = np.ones(len(avail_index))

    for f in program_prefs.columns:
        vals = fields[f][avail_index[:,0], avail_index[:,1], avail_index[:,2]]
        a_weighted_vals = np.where(np.isinf(vals), 0, vals ** agent_prefs[f])
        a_eval *= a_weighted_vals

    sorted_indices = np.argsort(-a_eval)
    for selected_int in sorted_indices:
        selected_ind = avail_index[selected_int]
        fns = avail_lattice.find_neighbours_masked(stencil_sq, loc=selected_ind)
        blocked = np.sum(occ_lattice[np.unravel_index(fns, avail_lattice.shape)] != -1)


        if blocked < 2:
            return selected_ind

    print(f"No suitable location found for agent {agent_id}")
    return None

In [13]:
# 기존 코드의 나머지 부분을 조정
occ_lattice = avail_lattice * 0 - 1
agn_num = len(program_complete)
agn_locs = [None] * agn_num

for a_id, a_prefs in program_complete.iterrows():
    agn_locs[a_id] = find_optimal_location_for_agent(a_id, a_prefs, avail_lattice, fields, stencil, occ_lattice)
    if agn_locs[a_id] is not None:
        avail_lattice[tuple(agn_locs[a_id])] = 0
        occ_lattice[tuple(agn_locs[a_id])] = a_id

# Calculate best starting point
# The order to calculate is based on their sizes
agn_locs = [None]*agn_num
for a_id, a_prefs in program_complete.iterrows():
    avail_index = np.array(np.where(avail_lattice)).T
    a_eval = np.ones(len(avail_index))
    for f in program_prefs.columns:
        vals = fields[f][avail_index[:,0], avail_index[:,1], avail_index[:,2]]
        # raise the the raw value to the power of preference weight of the agent
        a_weighted_vals = vals ** a_prefs[f]
        # multiply them to the previous weighted values
        a_eval *= a_weighted_vals
    for i in range(len(a_eval)):
        if a_eval[i] == np.inf:
            a_eval[i] = -1

        selected_int = np.argmax(a_eval)
    selected_ind = avail_index[selected_int]

    sorted_indices = np.argsort(-a_eval, axis=0)
    max_attempts = len(sorted_indices)  # maximum number of attempts
    attempt = 0

    while attempt < max_attempts:
        fns = avail_lattice.find_neighbours_masked(stencil, loc=selected_ind)
        blocked = sum(occ_lattice[np.unravel_index(n, avail_lattice.shape)] != -1 for n in fns)

        if blocked < 2:
            break

        selected_int = sorted_indices[attempt]
        selected_ind = avail_index[selected_int]
        attempt += 1

    if attempt == max_attempts:
        print("No suitable location found for agent", a_id)
        continue  # Or handle this situation as needed

    agn_locs[a_id] = [selected_ind]
    avail_lattice[tuple(selected_ind)] = 0
    occ_lattice[tuple(selected_ind)] = a_id

    lattice_cens = init_avail_lattice.centroids_threshold(-1)

In [14]:
from matplotlib.colors import ListedColormap
p = pv.Plotter(notebook=True)

# Set the grid dimensions
grid = pv.ImageData()
grid.dimensions = np.array(occ_lattice.shape) + 1
grid.origin = occ_lattice.minbound - occ_lattice.unit * 0.5
grid.spacing = occ_lattice.unit

# Adding the bounding box wireframe
p.add_mesh(grid.outline(), color="grey", label="Domain")
p.add_axes()
p.show_bounds(grid="back", location="back", color="#777777")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=0.6,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)
annotations = {i: name for i, name in enumerate(space_list.values())}
# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, annotations=annotations, scalar_bar_args=sargs, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시



Widget(value='<iframe src="http://localhost:61875/index.html?ui=P_0x1de6903d4d0_1&reconnect=auto" class="pyvis…

In [15]:
def distance(a_id, fns):
    fns_cens = lattice_cens[np.ravel_multi_index(np.array(fns).T, avail_lattice.shape)] # 이웃하는 격자의 중심점을 가져옴 (벡터화된 연산으로 개선)
    agn_avg_loc = np.average(agn_locs[a_id], axis=0)                                    # 에이전트의 평균 위치 계산 (루프 밖에서 한 번만 계산)
    dist_m = np.sqrt(((fns_cens - agn_avg_loc)**2).sum(axis=1))                         # 평균 위치와의 차이를 기반으로 거리 계산 (벡터화된 연산으로 개선)

    return dist_m  # 계산된 거리 반환

In [16]:
square_weight = 0.4

cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
frames = [cur_occ_lattice]

t = 0
n_frames=20

# 메인 루프: 각 시간 단계에 대해 반복
while t < n_frames:
    n_fns = 0

    for a_id, a_prefs in program_prefs.iterrows():
        # 각 에이전트에 대한 루프

        a_locs = agn_locs[a_id]
        free_neighs = []  # 빈 이웃 목록 초기화
        free_neighs_sq = []

        for loc in a_locs:
            # 에이전트의 각 위치에 대한 루프
            neighs = avail_lattice.find_neighbours_masked(stencil, loc=loc)
            neighs_sq = avail_lattice.find_neighbours_masked(stencil_sq, loc=loc)

            for n in neighs:
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                if avail_lattice[neigh_3d_id]:
                    free_neighs.append(neigh_3d_id)

            for n in neighs_sq:
                neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                if avail_lattice[neigh_3d_id]:
                    free_neighs_sq.append(neigh_3d_id)

        if not len(free_neighs) > 0:
            n_fns += 1

        if len(free_neighs) > 0:
            fns = np.array(free_neighs)

            a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

            for s in program_mtx.columns:
                s = int(s)
                vals = distance(s, fns)
                a_weighted_vals = vals ** program_mtx.loc[a_id, s]
                a_eval *= a_weighted_vals

            free_neighs_count = []

            for free_neigh in free_neighs:
                free_neighs_count.append(free_neighs_sq.count(free_neigh))

            a_weighted_square = np.array(free_neighs_count) ** square_weight
            a_eval *= a_weighted_square

            current_length = np.copy(len(a_locs))

            i_eval = np.zeros(current_length)
            max_space_raw = agent_areas[a_id]
            max_space = np.rint(max_space_raw)

            if current_length >= max_space:
                i_eval = np.ones(current_length)

                for f in program_prefs.columns:
                    vals = fields[f][np.array(a_locs)[:, 0], np.array(a_locs)[:, 1], np.array(a_locs)[:, 2]]
                    a_weighted_vals = vals ** a_prefs[f]
                    i_eval *= a_weighted_vals

                for s in program_mtx.columns:
                    s = int(s)
                    vals = distance(s, np.array(a_locs))
                    a_weighted_vals = vals ** program_mtx.loc[a_id, s]
                    i_eval *= a_weighted_vals

                i_neighs_count = np.zeros(current_length)

                for id, loc in enumerate(a_locs):
                    neighs = init_avail_lattice.find_neighbours_masked(stencil, loc=loc)
                    for n in neighs:
                        neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                        i_neighs_count[id] += (occ_lattice == a_id)[neigh_3d_id]

                i_weighted_square = np.array(i_neighs_count) ** square_weight
                i_eval *= i_weighted_square

            selected_int_inner = np.argmin(i_eval)
            selected_int = np.argmax(a_eval)

            if (current_length >= max_space) and i_eval[selected_int_inner] < a_eval[selected_int]:
                selected_inner_3d_id = tuple(a_locs[selected_int_inner])
                selected_inner_loc = a_locs[selected_int_inner]
                agn_locs[a_id].pop(selected_int_inner)
                avail_lattice[selected_inner_3d_id] = 1
                occ_lattice[selected_inner_3d_id] = -1

            if current_length < max_space:
                selected_neigh_3d_id = free_neighs[selected_int]
                selected_neigh_loc = np.array(selected_neigh_3d_id).flatten()
                agn_locs[a_id].append(selected_neigh_loc)
                avail_lattice[selected_neigh_3d_id] = 0
                occ_lattice[selected_neigh_3d_id] = a_id

    new_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
    frames.append(new_occ_lattice)
    print(t, "/", n_fns, end="  ")
    t += 1

    # for i, lattice in enumerate(frames):
    #     save_csv_path = os.path.relpath('/content/drive/MyDrive/topotry2/abm_mcda/abm_f_'+ f'{i:03}' + '.csv')
    #     lattice.to_csv(save_csv_path)

0 / 2  1 / 3  2 / 3  3 / 3  4 / 3  5 / 3  6 / 3  7 / 3  8 / 3  9 / 3  10 / 3  11 / 3  12 / 3  13 / 3  14 / 3  15 / 3  16 / 3  17 / 3  18 / 3  19 / 2  

In [17]:
import numpy as np
import pyvista as pv
from matplotlib.colors import ListedColormap
import ipywidgets as widgets
from IPython.display import display
import topogenesis as tg

square_weight = 0.5

cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
frames = [cur_occ_lattice]

t = 0
n_frames = 500

# Precompute max_space for each agent to avoid recalculating in loop
max_spaces = np.rint(agent_areas).astype(int)

while t < n_frames:
    n_fns = 0

    for a_id, a_prefs in program_prefs.iterrows():
        a_locs = agn_locs[a_id]
        free_neighs = []
        free_neighs_sq = []

        for loc in a_locs:
            neighs = avail_lattice.find_neighbours_masked(stencil, loc=loc)
            neighs_sq = avail_lattice.find_neighbours_masked(stencil_sq, loc=loc)

            free_neighs.extend([np.unravel_index(n, avail_lattice.shape) for n in neighs if avail_lattice[np.unravel_index(n, avail_lattice.shape)]])
            free_neighs_sq.extend([np.unravel_index(n, avail_lattice.shape) for n in neighs_sq if avail_lattice[np.unravel_index(n, avail_lattice.shape)]])

        if not free_neighs:
            n_fns += 1
            continue

        fns = np.array(free_neighs)
        a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

        for s in program_mtx.columns:
            s = int(s)
            a_eval *= distance(s, fns) ** program_mtx.loc[a_id, s]

        free_neighs_count = np.array([free_neighs_sq.count(tuple(free_neigh)) for free_neigh in free_neighs])
        a_eval *= free_neighs_count ** square_weight

        current_length = len(a_locs)
        max_space = max_spaces[a_id]

        if current_length >= max_space:
            i_eval = np.ones(current_length)
            a_locs_arr = np.array(a_locs)
            for f in program_prefs.columns:
                vals = fields[f][a_locs_arr[:, 0], a_locs_arr[:, 1], a_locs_arr[:, 2]]
                i_eval *= vals ** a_prefs[f]

            for s in program_mtx.columns:
                s = int(s)
                i_eval *= distance(s, a_locs_arr) ** program_mtx.loc[a_id, s]

            i_neighs_count = np.zeros(current_length)
            for idx, loc in enumerate(a_locs):
                neighs = avail_lattice.find_neighbours_masked(stencil, loc=loc)
                for n in neighs:
                    neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                    i_neighs_count[idx] += (occ_lattice == a_id)[neigh_3d_id]

            i_weighted_square = i_neighs_count ** square_weight
            i_eval *= i_weighted_square

            selected_int_inner = np.argmin(i_eval)
            selected_int = np.argmax(a_eval)

            if i_eval[selected_int_inner] < a_eval[selected_int]:
                selected_inner_loc = a_locs.pop(selected_int_inner)
                avail_lattice[tuple(selected_inner_loc)] = 1
                occ_lattice[tuple(selected_inner_loc)] = -1

        if current_length < max_space:
            selected_neigh_loc = free_neighs[np.argmax(a_eval)]
            agn_locs[a_id].append(list(selected_neigh_loc))
            avail_lattice[tuple(selected_neigh_loc)] = 0
            occ_lattice[tuple(selected_neigh_loc)] = a_id

    frames.append(tg.to_lattice(np.copy(occ_lattice), occ_lattice))
    print(t, "/", n_fns, end="  ")
    t += 1



In [18]:
square_weight = 1.1

cur_occ_lattice = tg.to_lattice(np.copy(occ_lattice), occ_lattice)
frames = [cur_occ_lattice]

t = 0
n_frames = 20

# Precompute max_space for each agent to avoid recalculating in loop
max_spaces = np.rint(agent_areas).astype(int)

# Cache the neighbors for each position in the lattice to avoid recalculating
neighbors_cache = {}
neighbors_sq_cache = {}
for x in range(avail_lattice.shape[0]):
    for y in range(avail_lattice.shape[1]):
        for z in range(avail_lattice.shape[2]):
            loc = (x, y, z)
            neighbors_cache[loc] = avail_lattice.find_neighbours_masked(stencil, loc=loc)
            neighbors_sq_cache[loc] = avail_lattice.find_neighbours_masked(stencil_sq, loc=loc)

while t < n_frames:
    n_fns = 0

    for a_id, a_prefs in program_prefs.iterrows():
        a_locs = agn_locs[a_id]
        free_neighs = []
        free_neighs_sq = []

        for loc in a_locs:
            neighs = neighbors_cache[tuple(loc)]
            neighs_sq = neighbors_sq_cache[tuple(loc)]

            free_neighs.extend([np.unravel_index(n, avail_lattice.shape) for n in neighs if avail_lattice[np.unravel_index(n, avail_lattice.shape)]])
            free_neighs_sq.extend([np.unravel_index(n, avail_lattice.shape) for n in neighs_sq if avail_lattice[np.unravel_index(n, avail_lattice.shape)]])

        if not free_neighs:
            n_fns += 1
            continue

        fns = np.array(free_neighs)
        a_eval = np.prod([fields[f][fns[:, 0], fns[:, 1], fns[:, 2]] ** a_prefs[f] for f in program_prefs.columns], axis=0)

        for s in program_mtx.columns:
            s = int(s)
            a_eval *= distance(s, fns) ** program_mtx.loc[a_id, s]

        free_neighs_count = np.array([free_neighs_sq.count(tuple(free_neigh)) for free_neigh in free_neighs])
        a_eval *= free_neighs_count ** square_weight

        current_length = len(a_locs)
        max_space = max_spaces[a_id]

        if current_length >= max_space:
            i_eval = np.ones(current_length)
            a_locs_arr = np.array(a_locs)
            for f in program_prefs.columns:
                vals = fields[f][a_locs_arr[:, 0], a_locs_arr[:, 1], a_locs_arr[:, 2]]
                i_eval *= vals ** a_prefs[f]

            for s in program_mtx.columns:
                s = int(s)
                i_eval *= distance(s, a_locs_arr) ** program_mtx.loc[a_id, s]

            i_neighs_count = np.zeros(current_length)
            for idx, loc in enumerate(a_locs):
                neighs = neighbors_cache[tuple(loc)]
                for n in neighs:
                    neigh_3d_id = np.unravel_index(n, avail_lattice.shape)
                    i_neighs_count[idx] += (occ_lattice == a_id)[neigh_3d_id]

            i_weighted_square = i_neighs_count ** square_weight
            i_eval *= i_weighted_square

            selected_int_inner = np.argmin(i_eval)
            selected_int = np.argmax(a_eval)

            if i_eval[selected_int_inner] < a_eval[selected_int]:
                selected_inner_loc = a_locs.pop(selected_int_inner)
                avail_lattice[tuple(selected_inner_loc)] = 1
                occ_lattice[tuple(selected_inner_loc)] = -1

        if current_length < max_space:
            selected_neigh_loc = free_neighs[np.argmax(a_eval)]
            agn_locs[a_id].append(list(selected_neigh_loc))
            avail_lattice[tuple(selected_neigh_loc)] = 0
            occ_lattice[tuple(selected_neigh_loc)] = a_id

    frames.append(tg.to_lattice(np.copy(occ_lattice), occ_lattice))
    print(t, "/", n_fns, end="  ")
    t += 1


In [19]:
# Initialize PyVista plotter
p = pv.Plotter(notebook=True)

# Setup the grid based on the lattice
base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(base_lattice.shape) + 1
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
grid.spacing = base_lattice.unit
p.add_mesh(grid.outline(), color="grey", label="Domain")

# Add the data values to the cell data
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)  # Flatten the array!

# Filtering the voxels
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

colors = [
    "#FFB6C1", "#FFC0CB", "#FF69B4", "#FF1493",  # 분홍색 계열
    "#87CEEB", "#00BFFF", "#1E90FF",  # 파란색 계열
    "#98FB98", "#00FF7F", "#32CD32", "#3CB371", "#2E8B57", "#228B22",  # 녹색 계열
    "#FFD700", "#FFA500", "#FF8C00", "#FF7F50", "#FF6347", "#FF4500", "#FF0000", "#DC143C", "#B22222",  # 노란색 계열
    "#BA55D3", "#9932CC", "#9400D3", "#8A2BE2", "#9370DB", "#7B68EE", "#6A5ACD",  # 자주색 계열
    "#FFE4B5", "#FFDAB9", "#FFDEAD",  # 베이지 계열
    "#F0E68C", "#EEE8AA", "#F5DEB3", "#DEB887", "#D2B48C", "#BC8F8F",  # 갈색 계열
    "#B0C4DE", "#ADD8E6", "#B0E0E6", "#AFEEEE", "#00CED1",  # 연한 파란색 계열
    "#696969",  # 회색 계열
    "#708090",  # 슬레이트 그레이 계열
    "#4682B4", "#5F9EA0", "#6495ED", "#7B68EE", "#87CEFA"  # 추가 파란색 계열
]

custom_cmap = ListedColormap(colors)
# Scalar bar 설정
sargs = dict(
    shadow=True,
    n_labels=agn_num,
    italic=False,
    fmt=" %.0f",
    font_family="arial",
    height=1,
    vertical=True,

    title_font_size=10,  # 제목 폰트 크기 설정
    label_font_size=6   # 라벨 폰트 크기 설정
)

# Adding the voxels with the custom colormap and scalar bar arguments
p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=True, scalar_bar_args=sargs, annotations=annotations, cmap=custom_cmap)
street_pc.fast_notebook_vis(p)
#p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa', opacity=0.3)
p.add_mesh(entrance_pc, color='red', opacity=1.0)
#envelope_lattice.fast_vis(p)
# Show the plot
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시

Widget(value='<iframe src="http://localhost:61875/index.html?ui=P_0x1de6e6654d0_2&reconnect=auto" class="pyvis…

In [20]:
from matplotlib.colors import ListedColormap

# PyVista 플로터 초기화
p = pv.Plotter(notebook=True)

# 그리드 설정
base_lattice = frames[0]
grid = pv.ImageData()
grid.dimensions = np.array(base_lattice.shape) + 1
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
grid.spacing = base_lattice.unit
p.add_mesh(grid.outline(), color="grey", label="Domain")

# 데이터 할당
grid.cell_data["Agents"] = occ_lattice.flatten(order="F").astype(int)

# 필터링
agn_num = len(program_prefs)
threshed = grid.threshold([-0.1, agn_num - 0.9])

# 초기 컬러맵 설정
initial_colors = ["#FF0000" if i == 0 else "#808080" for i in range(len(program_prefs))]
custom_cmap = ListedColormap(initial_colors)

# 스칼라 바 설정
annotations = {i: name for i, name in enumerate(space_list.values())}
sargs = {
    "shadow": True,
    "n_labels": len(program_prefs),
    "italic": False,
    "fmt": " %.0f",
    "font_family": "arial",
    "height": 1,
    "vertical": True,
    "title_font_size": 10,
    "label_font_size": 6
}

# 메시 추가
opacity_map = np.full(threshed.n_cells, 0.1)  # 초기 투명도 설정
threshed.cell_data['opacity'] = opacity_map
mesh = p.add_mesh(threshed, scalars='opacity', cmap=custom_cmap, show_scalar_bar=True, scalar_bar_args=sargs, show_edges=True, opacity=1)

# 텍스트 추가
text_actor = p.add_text("Selected Program: None", position='upper_right', font_size=12, color='black')

# 슬라이더 콜백 함수
def slider_callback(value):
    selected_program = int(value)
    mask = threshed.cell_data['Agents'] == selected_program
    opacity = np.full(threshed.n_cells, 0.1)  # 기본 투명도 설정
    opacity[mask] = 1.0  # 선택된 프로그램에 높은 투명도 부여

    # 투명도 업데이트
    threshed.cell_data['opacity'] = opacity
    mesh.update_scalars(opacity, render=False)
    
    # 컬러맵 업데이트
    colors = ["#FF0000" if threshed.cell_data['Agents'][i] == selected_program else "#808080" for i in range(len(threshed.cell_data['Agents']))]
    mesh.point_data['colors'] = colors
    
    text_actor.SetText(0, f"Selected Program: {annotations[selected_program]}")
    p.render()

# 슬라이더 위젯 추가
p.add_slider_widget(slider_callback, [0, len(program_prefs) - 1], value=0, title='Program Index')

# 시각화 표시
p.show(jupyter_backend="trame", return_viewer=True)  # 시각화 표시


c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyvista\core\utilities\misc.py:207: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\junglim\AppData\Local\Temp\ipykernel_22708\2536417408.py", line 56, in slider_callback
    mesh.update_scalars(opacity, render=False)
    ^^^^^^^^^^^^^^^^^^^
AttributeError: 'Actor' object has no attribute 'update_scalars'
  warnings.warn(formatted_exception)


Widget(value='<iframe src="http://localhost:61875/index.html?ui=P_0x1de020944d0_3&reconnect=auto" class="pyvis…

c:\Users\junglim\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyvista\core\utilities\misc.py:207: UserWarning: Encountered issue in callback (most recent call last):
  File "C:\Users\junglim\AppData\Local\Temp\ipykernel_22708\2536417408.py", line 56, in slider_callback
    mesh.update_scalars(opacity, render=False)
    ^^^^^^^^^^^^^^^^^^^
AttributeError: 'Actor' object has no attribute 'update_scalars'
  warnings.warn(formatted_exception)
